In [1]:
import pandas as pd
import requests
import config

In [2]:
key = config.key # Personal Google Maps API key

In [2]:
subway_stations = pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')

#Split out, then combine, latitude and longitude
subway_stations['latitude'] = subway_stations['the_geom'].apply(lambda x: x.split()[2].replace(')',''))
subway_stations['longitude'] = subway_stations['the_geom'].apply(lambda x: x.split()[1].replace('(',''))
subway_stations['latitude&longitude'] = subway_stations['latitude']+' '+subway_stations['longitude']

                                          the_geom  \
0     POINT (-74.00030814706824 40.73225482650675)   
1    POINT (-73.83256899924748 40.846810332614844)   
2    POINT (-73.97192000013308 40.757107333148234)   
3     POINT (-73.97621799811347 40.78864433404891)   
4    POINT (-74.00413100111697 40.713065332984044)   
5      POINT (-73.98207600124924 40.7460813321576)   
6     POINT (-73.96796700066442 40.76252633389173)   
7     POINT (-73.85736239433544 40.89314357506239)   
8    POINT (-73.98220900041213 40.773440334231445)   
9     POINT (-73.89054899975386 40.82094833186627)   
10   POINT (-74.00627700023756 40.722854331339036)   
11   POINT (-73.83632199732654 40.843863334673635)   
12     POINT (-73.98659900183978 40.7398643335507)   
13   POINT (-73.94526400065335 40.747023331863446)   
14    POINT (-73.99106999895504 40.73005433376933)   
15    POINT (-73.98192900168259 40.76824733393756)   
16     POINT (-73.9489160011607 40.74221633326165)   
17   POINT (-73.951070000706

In [9]:
subway_stations

,Unnamed: 0,the_geom,NAME,URL,LINE,latitude,longitude,latitude&longitude,Central Park,The High Line,...,Bronx Zoo,American Museum of Natural History,Flushing Main Street,"Whitney+Museum+of+American+Art,+99+Gansevoort+St,+New+York,+NY+10014","The+Met+Cloisters,+99+Margaret+Corbin+Dr,+New+York,+NY+10040",avg_distance,max_distance,min_distance,"Chelsea+Market,+75+9th+Ave,+New+York,+NY+10011",med_distance
0,359,POINT (-73.98749500043364 40.75529033316711),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,1/2/2003,40.755290,-73.987495,40.75529033316711 -73.98749500043364,18.933333,14.300000,...,50.150000,13.300000,45.900000,15.000000,45.150000,19.968254,50.150000,3.416667,12.883333,15.000000
1,362,POINT (-73.98973500043815 40.75730833180808),42nd St - Port Authority Bus Term,http://www.mta.info/nyct/subway/index.html,A-C-E,40.757308,-73.989735,40.75730833180808 -73.98973500043815,14.450000,10.433333,...,54.916667,8.816667,50.666667,13.650000,40.666667,20.551587,54.916667,5.983333,9.400000,14.450000
2,154,POINT (-73.98769099766001 40.75547733499068),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,7-7 Express,40.755477,-73.987691,40.75547733499068 -73.98769099766001,16.916667,14.300000,...,50.150000,14.433333,55.900000,16.000000,45.133333,20.606349,55.900000,3.566667,14.866667,16.000000
3,366,POINT (-73.98777189015544 40.74978973294837),Herald Sq - 34th St,http://www.mta.info/nyct/subway/index.html,B-D-F-M,40.749790,-73.987772,40.74978973294837 -73.98777189015544,18.700000,16.216667,...,56.316667,16.600000,58.100000,19.483333,48.450000,20.911905,58.100000,3.050000,14.183333,18.216667
4,88,POINT (-73.986768002027 40.754612332230856),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,N-Q-R,40.754612,-73.986768,40.754612332230856 -73.986768002027,18.250000,12.816667,...,51.483333,15.766667,57.233333,16.333333,45.350000,20.976984,57.233333,3.216667,16.066667,16.333333
5,19,POINT (-73.99565700196479 40.74408133288406),23rd St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.744081,-73.995657,40.74408133288406 -73.99565700196479,21.633333,11.066667,...,54.150000,16.000000,58.383333,12.000000,48.850000,21.163492,58.383333,6.716667,10.583333,15.533333
6,160,POINT (-73.99336500139277 40.74721533235655),28th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.747215,-73.993365,40.74721533235655 -73.99336500139277,19.916667,11.183333,...,53.150000,19.016667,47.283333,13.000000,50.866667,21.391270,53.150000,6.083333,12.266667,15.900000
7,83,POINT (-73.98793683324304 40.749644893078916),Herald Sq - 34th St,http://www.mta.info/nyct/subway/index.html,N-Q-R,40.749645,-73.987937,40.749644893078916 -73.98793683324304,19.216667,16.550000,...,56.333333,20.616667,49.100000,18.183333,48.783333,21.403968,56.333333,3.383333,14.516667,18.550000
8,28,POINT (-73.98384899991842 40.76172833308103),50th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.761728,-73.983849,40.76172833308103 -73.98384899991842,15.400000,15.383333,...,49.000000,9.766667,56.600000,17.850000,42.616667,21.474603,56.600000,4.266667,15.350000,15.400000
9,164,POINT (-73.99787100112236 40.741040332966726),18th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.741040,-73.997871,40.741040332966726 -73.99787100112236,23.733333,10.766667,...,55.150000,18.100000,52.900000,11.000000,49.950000,21.576984,55.150000,5.716667,10.266667,17.016667


In [18]:
def get_duration_in_minutes(origin, destination, mode='transit', key=key):
    """Return duration in minutes between origin and destination using specified mode of transport.
    Use Google Maps Direction API
    """
    url = 'https://maps.googleapis.com/maps/api/directions/json?origin='+origin+'&destination='+destination+'&mode=transit&transit_mode=rail&key='+key
    # Request directions via public transit
    result = requests.get(url)
    directions = result.json()
    try:
        return directions['routes'][0]['legs'][0]['duration']['value']/60.0
    except:
        print(directions)

def get_durations(df, destinations):
    """Return duration between locations for all rows in dataframe and all destinations"""
    for destination in destinations:
        df[destination] = 0
        
    for origin in df['latitude&longitude']:
        print(origin)
        for destination in destinations:
            print(destination)
            df.loc[df['latitude&longitude']==origin, destination] = get_duration_in_minutes(origin, destination)
            
    return df

def aggregate_durations(df, destinations):
    """Calculate basic statistics about durations in dataframe"""
    df['avg_duration'] = df[destinations].mean(axis=1)
    df['max_duration'] = df[destinations].max(axis=1)
    df['min_duration'] = df[destinations].min(axis=1)
    df['med_duration'] = df[destinations].median(axis=1)
    return df

def replace(df, old, new):
    """Replace durations for old destination with durations for new destination""" 
    df.rename(columns = {old:new}, inplace = True)
    
    for i in range(df['latitude&longitude'].size):
        origin = df['latitude&longitude'][i]
        print(origin)
        destination = new
        url = 'https://maps.googleapis.com/maps/api/directions/json?origin='+origin+'&destination='+destination+'&mode=transit&transit_mode=rail&key='+key
        # Request directions via public transit
        result = requests.get(url)
        directions = result.json()
        df.loc[i, destination] = directions['routes'][0]['legs'][0]['duration']['value']/60.0
    
    return df

destinations = ['Central Park', 'The High Line', '9/11 Memorial', 'Statue of Liberty', \
                'Museum of Modern Art, 11 W 53rd St, New York, NY 10019'.replace(' ', '+'), \
                'Top of the Rock, 30 Rockefeller Plaza, New York, NY 10111'.replace(' ', '+'), \
                'The Frick Collection, 1 E 70th St, New York, NY 10021'.replace(' ', '+'), \
                'Bryant Park', \
                'Grand Central Terminal', 'Times Square', 'Brooklyn Bridge', 'West Village', 'Empire State Building', \
                'New York Transit Museum, 99 Schermerhorn St, Brooklyn, NY 11201'.replace(' ', '+'), 'Brooklyn Botanic Garden', 'Bronx Zoo', \
                'American Museum of Natural History', 'Flushing Main Street', \
                'Whitney Museum of American Art, 99 Gansevoort St, New York, NY 10014'.replace(' ', '+'), \
                'Chelsea Market, 75 9th Ave, New York, NY 10011'.replace(' ', '+'), \
                'The Met Cloisters, 99 Margaret Corbin Dr, New York, NY 10040'.replace(' ', '+')]

In [ ]:
subway_stations = get_durations(subway_stations, destinations)

In [ ]:
subway_stations = aggregate_durations(subway_stations, destinations)

In [13]:
subway_stations.sort_values(by='avg_duration', inplace=True)
print(subway_stations[['NAME', 'LINE', 'avg_duration']].head(50))
subway_stations.to_csv('subway_stations.csv')

                                 NAME             LINE  avg_duration
0                  Times Sq - 42nd St         1/2/2003     19.968254
1   42nd St - Port Authority Bus Term            A-C-E     20.551587
2                  Times Sq - 42nd St      7-7 Express     20.606349
3                 Herald Sq - 34th St          B-D-F-M     20.911905
4                  Times Sq - 42nd St            N-Q-R     20.976984
5                             23rd St            2-Jan     21.163492
6                             28th St            2-Jan     21.391270
7                 Herald Sq - 34th St            N-Q-R     21.403968
8                             50th St            2-Jan     21.474603
9                             18th St            2-Jan     21.576984
10             34th St - Penn Station         1/2/2003     21.677778
11                 Times Sq - 42nd St                S     21.742857
12                            14th St            A-C-E     21.806349
13                            23rd

In [14]:
subway_stations

,Unnamed: 0,the_geom,NAME,URL,LINE,latitude,longitude,latitude&longitude,Central Park,The High Line,...,Bronx Zoo,American Museum of Natural History,Flushing Main Street,"Whitney+Museum+of+American+Art,+99+Gansevoort+St,+New+York,+NY+10014","The+Met+Cloisters,+99+Margaret+Corbin+Dr,+New+York,+NY+10040",avg_duration,max_duration,min_duration,"Chelsea+Market,+75+9th+Ave,+New+York,+NY+10011",med_duration
0,359,POINT (-73.98749500043364 40.75529033316711),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,1/2/2003,40.755290,-73.987495,40.75529033316711 -73.98749500043364,18.933333,14.300000,...,50.150000,13.300000,45.900000,15.000000,45.150000,19.968254,50.150000,3.416667,12.883333,15.000000
1,362,POINT (-73.98973500043815 40.75730833180808),42nd St - Port Authority Bus Term,http://www.mta.info/nyct/subway/index.html,A-C-E,40.757308,-73.989735,40.75730833180808 -73.98973500043815,14.450000,10.433333,...,54.916667,8.816667,50.666667,13.650000,40.666667,20.551587,54.916667,5.983333,9.400000,14.450000
2,154,POINT (-73.98769099766001 40.75547733499068),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,7-7 Express,40.755477,-73.987691,40.75547733499068 -73.98769099766001,16.916667,14.300000,...,50.150000,14.433333,55.900000,16.000000,45.133333,20.606349,55.900000,3.566667,14.866667,16.000000
3,366,POINT (-73.98777189015544 40.74978973294837),Herald Sq - 34th St,http://www.mta.info/nyct/subway/index.html,B-D-F-M,40.749790,-73.987772,40.74978973294837 -73.98777189015544,18.700000,16.216667,...,56.316667,16.600000,58.100000,19.483333,48.450000,20.911905,58.100000,3.050000,14.183333,18.216667
4,88,POINT (-73.986768002027 40.754612332230856),Times Sq - 42nd St,http://www.mta.info/nyct/subway/index.html,N-Q-R,40.754612,-73.986768,40.754612332230856 -73.986768002027,18.250000,12.816667,...,51.483333,15.766667,57.233333,16.333333,45.350000,20.976984,57.233333,3.216667,16.066667,16.333333
5,19,POINT (-73.99565700196479 40.74408133288406),23rd St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.744081,-73.995657,40.74408133288406 -73.99565700196479,21.633333,11.066667,...,54.150000,16.000000,58.383333,12.000000,48.850000,21.163492,58.383333,6.716667,10.583333,15.533333
6,160,POINT (-73.99336500139277 40.74721533235655),28th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.747215,-73.993365,40.74721533235655 -73.99336500139277,19.916667,11.183333,...,53.150000,19.016667,47.283333,13.000000,50.866667,21.391270,53.150000,6.083333,12.266667,15.900000
7,83,POINT (-73.98793683324304 40.749644893078916),Herald Sq - 34th St,http://www.mta.info/nyct/subway/index.html,N-Q-R,40.749645,-73.987937,40.749644893078916 -73.98793683324304,19.216667,16.550000,...,56.333333,20.616667,49.100000,18.183333,48.783333,21.403968,56.333333,3.383333,14.516667,18.550000
8,28,POINT (-73.98384899991842 40.76172833308103),50th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.761728,-73.983849,40.76172833308103 -73.98384899991842,15.400000,15.383333,...,49.000000,9.766667,56.600000,17.850000,42.616667,21.474603,56.600000,4.266667,15.350000,15.400000
9,164,POINT (-73.99787100112236 40.741040332966726),18th St,http://www.mta.info/nyct/subway/index.html,2-Jan,40.741040,-73.997871,40.741040332966726 -73.99787100112236,23.733333,10.766667,...,55.150000,18.100000,52.900000,11.000000,49.950000,21.576984,55.150000,5.716667,10.266667,17.016667


In [19]:
subway_stations[destinations].idxmax(axis=1)

0                                              Bronx Zoo
1                                              Bronx Zoo
2                                   Flushing Main Street
3                                   Flushing Main Street
4                                   Flushing Main Street
5                                   Flushing Main Street
6                                              Bronx Zoo
7                                              Bronx Zoo
8                                   Flushing Main Street
9                                              Bronx Zoo
10                                  Flushing Main Street
11                                  Flushing Main Street
12                                  Flushing Main Street
13                                  Flushing Main Street
14                                  Flushing Main Street
15                                             Bronx Zoo
16                                  Flushing Main Street
17                             

In [20]:
subway_stations.groupby(by='LINE')['avg_duration'].mean().sort_values()

LINE
F-M                22.102381
A-C-E              22.500000
B-D-F-M            22.659841
2-Jan              23.303175
N-Q-R              23.510544
B-D-E              23.704762
1/2/2003           23.960450
4-5-6-6 Express    27.626587
A-C-F              27.642857
4-6-6 Express      27.788307
E-M                28.730159
A-B-C-D            29.812169
5-Apr              30.209048
7-7 Express-N-W    30.934921
3-Feb              31.245397
A-B-C              31.280853
A-C-G              31.973810
N-R                32.297052
B-D-N-Q-R          33.234921
S                  35.810000
2-3-4-5            36.702116
7-7 Express        37.004603
N-Q                37.303741
2/3/2004           39.594974
A-C                39.967725
J-M-Z              40.097619
1                  41.105992
D-N-R              41.672090
E                  41.803968
E-F-M-R            42.269841
                     ...    
L                  44.452249
B-D                44.563420
E-M-R              44.662121
3        